## 👨🏻‍💻 Практика - Сравнение моделей 
Использование метрик качества для сравнения производительности различных моделей машинного обучения.

Выбор солвера: Экспериментируйте с разными солверами для логистической регрессии, чтобы найти наиболее подходящий для ваших данных.

Визуализация результатов: Визуализируйте результаты сравнения моделей (например, с помощью боксплотов для кросс-валидации или ROC-кривых для классификации), чтобы сделать вывод более наглядным.

✔️**Решение:**
Использован датасет [sales.csv](./files/sales.csv)
### ✔ Импорт библиотек: 
Импортируем все необходимые библиотеки

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, make_scorer

In [2]:
# 1. Загрузка и подготовка данных (замените 'data.csv' на имя вашего файла)
data = pd.read_csv(r'C:\Users\79181\n.brykovskaya\module_2\4ML\1Introduction_ML\files\sales.csv')
data.head()

,sales,ad_spend,stores
0,349.998247,43.708611,36
1,485.616900,95.564288,43
2,432.109601,75.879455,36
3,246.479411,63.879264,8
4,272.338906,24.041678,34


In [3]:
# 2. Разделение на признаки и целевую переменную
# Предположим, что 'stores' - целевая переменная
X = data.drop('stores', axis=1)
y = data['stores']

In [4]:
# 3. Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = (
    train_test_split(X, y, test_size=0.2, random_state=42)
)

In [5]:
# 4. Определение моделей и гиперпараметров для перебора
models = [
    {
        'name': 'Linear Regression',
        'model': LinearRegression(),
        'params': {
           'fit_intercept': [True, False]
        }
    },
    {
        'name': 'Logistic Regression',
        'model': LogisticRegression(),
        'params': {
            'penalty': ['l1', 'l2'],
            'C': [0.1, 1, 10],
            'solver': ['liblinear', 'saga']  # Добавлено несколько вариантов солверов
        }
    }
]

In [6]:
# 5. Перекрестная проверка и оценка моделей
results = []
for model_config in models:
    print(f"Оценка модели: {model_config['name']}")
    # Определение метрики качества
    if model_config['name'] == 'Linear Regression':
        scoring = 'neg_mean_squared_error' # Используем встроенный негативный MSE
    else:
        scoring = 'accuracy'

    # GridSearchCV для перебора гиперпараметров и кросс-валидации
    grid_search = GridSearchCV(
        estimator=model_config['model'],
        param_grid=model_config['params'],
        scoring=scoring,
        cv=5,  # 5-fold cross-validation
        n_jobs=-1  # Использовать все доступные ядра процессора
    )
    grid_search.fit(X_train, y_train)

    # Сохранение результатов
    results.append({
        'model': model_config['name'],
        'best_params': grid_search.best_params_,
        'best_score': grid_search.best_score_,
        'best_estimator': grid_search.best_estimator_
    })

    # Вывод результатов
    print(f"Лучшие параметры: {grid_search.best_params_}")
    print(f"Лучший результат (кросс-валидация): {grid_search.best_score_}")
    print("-" * 30)

Оценка модели: Linear Regression
Лучшие параметры: {'fit_intercept': True}
Лучший результат (кросс-валидация): -14.916362197545851
------------------------------
Оценка модели: Logistic Regression
Лучшие параметры: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Лучший результат (кросс-валидация): 0.125
------------------------------


C:\Users\79181\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
C:\Users\79181\anaconda3\Lib\site-packages\sklearn\svm\_base.py:1249: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [7]:
# 6. Выбор лучшей модели (сравнение результатов)
best_model = max(results, key=lambda x: x['best_score'])

print("Лучшая модель:")
print(f"Модель: {best_model['model']}")
print(f"Параметры: {best_model['best_params']}")
print(f"Результат: {best_model['best_score']}")

Лучшая модель:
Модель: Logistic Regression
Параметры: {'C': 10, 'penalty': 'l1', 'solver': 'liblinear'}
Результат: 0.125


In [8]:
# Оценка на тестовой выборке
best_estimator = best_model['best_estimator']
y_pred = best_estimator.predict(X_test)

if best_model['model'] == 'Linear Regression':
    mse = mean_squared_error(y_test, y_pred)
    print(f"MSE (тестовая выборка): {mse}")
else:
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy (тестовая выборка): {accuracy}")

Accuracy (тестовая выборка): 0.15
